In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [17]:
import sys
sys.path.append("..")

import argparse
import scipy
import torch
import time
import torch.nn as nn
import os
import pandas as pd
import seaborn as sns
import numpy as np
import pickle
import yaml
import matplotlib.pyplot as plt
import torch.nn.functional as F

import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
import pywt
import math

from fno_utils import FNO2d, FNODatasetSingle

In [18]:
device = "cuda"

cfg_fn = os.path.join("..", "experiments", f"config_rdb.yaml")
with open(cfg_fn, "r") as f:
    cfg = yaml.safe_load(f)

In [19]:
pde_name = cfg["filename"].split(".h")[0]
model_weights = torch.load(os.path.join("..", "experiments", f"{pde_name}_FNO.pt"), map_location=torch.device('cuda'))

fno = FNO2d(
    num_channels=cfg["num_channels"], 
    modes1=cfg["modes"], 
    modes2=cfg["modes"], 
    width=cfg["width"], 
    initial_step=cfg["initial_step"]).to("cuda")
fno.load_state_dict(model_weights["model_state_dict"])

<All keys matched successfully>

In [20]:
scores = []
downsampling = [1,2,4]
for resolution in downsampling:
    batch_size = 25
    train_data = FNODatasetSingle(filename=os.path.join("..", "experiments", cfg["filename"]), reduced_resolution=resolution)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)

    res_scores = []
    for xxbatch, yy, gridbatch in train_loader:
        if cfg["training_type"] == "autoregressive":
            inp_shape = list(xxbatch.shape)
            inp_shape = inp_shape[:-2]
            inp_shape.append(-1)

            xxbatch = xxbatch.reshape(inp_shape)
            yyhat   = fno(xxbatch.to(device), gridbatch.to(device))
            yybatch = yy[:,:,:,10:11,:].to(device)
        else:
            xidx = 0
            xx   = xxbatch[xidx:xidx+1,...].to(device)
            grid = gridbatch[xidx:xidx+1,...].to(device)
            yhat = fno(xx[...,0,:], grid)
        break
    break

.H5 file extension is assumed hereafter


In [13]:
import pickle

fn = "../results/regret_rdb-True.pkl"
with open(fn, "rb") as f:
    regrets = pickle.load(f)

In [6]:
# Fixing parameters
scale = 7
dx    = pow(2, -scale)
k_s   = 1. / sqrt(pow(2, scale))

# Wavelet decomposition
wavelet_family = "db2"
wavelet = pywt.Wavelet(wavelet_family)
fL, fH, x_wav = wavelet.wavefun(level=scale+2)

A = 0.0105 # db2: ~0.0105
B = wavelet.rec_len - 2

def find_nearest(array, value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return idx-1
    return idx

def get_disc_idxs(xi, ks, inv_s):
    disc_xs = inv_s * xi - ks + B
    idx_tmp = np.searchsorted(x_wav, disc_xs)
    idxs = np.clip(idx_tmp - 1, 0, len(x_wav)-1)
    return idxs

def cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)

In [7]:
y = yyhat[0,...,0,0].cpu().detach().numpy()

coeffs = pywt.dwt2(y, wavelet_family)
nb_levels = len(coeffs) - 1

LL, (LH, HL, HH) = coeffs
coeffs = np.array([LL, LH, HL, HH]).reshape(-1)
coeffs = np.expand_dims(np.expand_dims(coeffs, axis=-1), axis=-1)

In [8]:
x_rec  = np.arange(0, 1, dx)
x_grid = cartesian_product(x_rec, x_rec)
x_grid_offset = np.expand_dims(np.expand_dims(x_grid - A, 1), 1)

inv_s = pow(2, scale - nb_levels)
ks    = np.array([[(k_1, k_2) for k_2 in range(LL.shape[1])] for k_1 in range(LL.shape[0])])
idxs  = get_disc_idxs(x_grid_offset, ks, inv_s)

final_shape_tuple = (x_rec.shape[0], x_rec.shape[0], -1)

fL_x, fL_y = fL[idxs[...,0]].reshape(final_shape_tuple), fL[idxs[...,1]].reshape(final_shape_tuple)
fH_x, fH_y = fH[idxs[...,0]].reshape(final_shape_tuple), fH[idxs[...,1]].reshape(final_shape_tuple)

disc_fLL = (fL_x * fL_y).reshape(final_shape_tuple)
disc_fLH = (fH_x * fL_y).reshape(final_shape_tuple)
disc_fHL = (fL_x * fH_y).reshape(final_shape_tuple)
disc_fHH = (fH_x * fH_y).reshape(final_shape_tuple)
disc_f   = (k_s * sqrt(inv_s) / sqrt(2)) * np.concatenate([disc_fLL, disc_fLH, disc_fHL, disc_fHH], axis=-1)
disc_f   = np.transpose(disc_f, axes=(2, 0, 1))

In [9]:
y_rec = np.sum(disc_f * coeffs, axis=0)

In [56]:
def basin_integral(f, center, radius):
    integral_mask = (np.linalg.norm(x_grid - center, axis=1) < radius).astype(np.int8).reshape(y_rec.shape)
    return np.sum(f * integral_mask)

In [57]:
basin_center  = np.array([0.75, 0.6])
basin_radius  = 0.2
integral      = basin_integral(disc_f[0], basin_center, basin_radius)

In [49]:
# ---------------------------------
# yhat = yhat.reshape(1, -1)
# y    = yybatch[0, ...,0,0]

In [33]:
diff = y - yhat
diff = diff.reshape(diff.shape[0], -1).detach().cpu().numpy()
# np.linalg.norm(diff.reshape(diff.shape[0], -1), ord=2)

In [43]:
import cvxpy as cp
from wavelet import get_disc_grid

In [55]:
w_mask.shape

(16384,)

In [57]:
yhat = yyhat[0, ...,0,0].detach().cpu().numpy().flatten()
w = np.array([0.6, 0.6])
r = 0.05
quantile = 1.0

x_grid = get_disc_grid(128)
w_mask = (np.linalg.norm(x_grid - w, axis=1) < r).astype(np.int8)

u = cp.Variable(yhat.shape)

objective = cp.Minimize(-u @ w_mask)
constraints = [cp.norm(u - yhat) <= quantile]
prob = cp.Problem(objective, constraints)
obj  = prob.solve()

In [58]:
obj

-153.77769533509266